# Importing Libraries

In [1]:
! pip install biosppy   

     |████████████████████████████████| 85 kB 1.5 MB/s eta 0:00:011
  Created wheel for biosppy: filename=biosppy-0.7.3-py2.py3-none-any.whl size=95409 sha256=cf13a2e640dad03ecc380a26b4a9c948e26c0e29d920d95a2a31a7ffe79a35bc
  Stored in directory: /root/.cache/pip/wheels/2f/4f/8f/28b2adc462d7e37245507324f4817ce1c64ef2464f099f4f0b
Successfully built biosppy


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import wfdb
import os                                                                                                  
import gc
import scipy       
import sklearn
from pathlib import Path
from sklearn.utils import shuffle
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn import preprocessing
import shutil
import math
import random
from scipy.spatial import distance
from biosppy.signals import ecg
from scipy.interpolate import PchipInterpolator

In [3]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Running on TPU  ['10.0.0.2:8470']


# Dataset Creation

## MIT-BIH Dataset

In [ ]:
####### Dataset Creation  
###### Reading Dataset

##### Function to read a record from single dataset
def data_read_mit(filepath):
    
    """
    Function to read the the inputs from MIT-BIH Dataset
    INPUTS:- 
    1)filepath: Path to the csv file
    
    OUTPUTS:-
    1)Output_signal: Output 1D array signal 
    """
    return (np.array(pd.read_csv(filepath,index_col=0).iloc[:,[0]]))   
    
##### Function to Required Annotations from .txt files
def feature_extractor(txt_file_path):
    
    """
       Function to extract time series data 
       from a .txt file
    """
    
    #### Reading File
    line_list = []

    with open(txt_file_path, 'r') as reader:

    # Read and print the entire file line by line
        line = reader.readline()
        while line != '':  # The EOF char is an empty string
            #print(line, end='')
            line_list.append(line)
            line = reader.readline()
    
    #### Taking the Time Step Data
    line_list = line_list[1:]
    
    #### Splitting the Collected Text Strings and Converting them into Floating type values
    new_list = []

    for item in line_list:
        for idx,sub_item in enumerate(item.split()):
            if(idx == 1):
                new_list.append(int(sub_item))
                break
    
    ### Returning the feature extracted list as numpy array
    return np.array(new_list)

###### Function to Segment Signals
##### Constants
FS = 500
W_LEN = 256
W_LEN_1_4 = 256 // 4
W_LEN_3_4 = 3 * (256 // 4)

##### Function
def segmentSignals(signal, r_peaks_annot, normalization=True, person_id= None, file_id=None):
    
    """
    Segments signals based on the detected R-Peak
    Args:
        signal (numpy array): input signal
        r_peaks_annot (int []): r-peak locations.
        normalization (bool, optional): apply z-normalization or not? . Defaults to True.
        person_id ([type], optional): [description]. Defaults to None.
        file_id ([type], optional): [description]. Defaults to None.
    Returns:
            [tuple(numpy array,numpy array)]: segmented signals and refined r-peaks
    """
    def refine_rpeaks(signal, r_peaks):
        """
        Refines the detected R-peaks. If the R-peak is slightly shifted, this assigns the 
        highest point R-peak.
        Args:
            signal (numpy array): input signal
            r_peaks (int []): list of detected r-peaks
        Returns:
            [numpy array]: refined r-peaks
        """

        r_peaks2 = np.array(r_peaks)            # make a copy
        for i in range(len(r_peaks)):
            r = r_peaks[i]          # current R-peak
            small_segment = signal[max(0,r-100):min(len(signal),r+100)]         # consider the neighboring segment of R-peak
            r_peaks2[i] = np.argmax(small_segment) - 100 + r_peaks[i]           # picking the highest point
            r_peaks2[i] = min(r_peaks2[i],len(signal))                          # the detected R-peak shouldn't be outside the signal
            r_peaks2[i] = max(r_peaks2[i],0)                                    # checking if it goes before zero    
        return r_peaks2                     # returning the refined r-peak list
    
    segmented_signals = []                      # array containing the segmented beats
    
    r_peaks = np.array(r_peaks_annot)
    r_peaks = refine_rpeaks(signal, r_peaks)
    skip_len = 5 # Parameter to specify number of r_peaks in one signal
    max_seq_len = 1280 # Parameter to specify maximum sequence length
    
    for r_curr in range(0,int(r_peaks.shape[0]-(skip_len-1)),skip_len):
        if ((r_peaks[r_curr]-W_LEN_1_4)<0) or ((r_peaks[r_curr+(skip_len-1)]+W_LEN_3_4)>=len(signal)):           # not enough signal to segment
            continue
        segmented_signal = np.array(signal[r_peaks[r_curr]-W_LEN_1_4:r_peaks[r_curr+(skip_len-1)]+W_LEN_3_4])        # segmenting a heartbeat
        segmented_signal = list(segmented_signal)
        #print(segmented_signal.shape)
        
        if(len(segmented_signal) < 1280):
            for m in range(int(1280-len(segmented_signal))): # Zero Padding
                segmented_signal.append(0)
        else:
            segmented_signal = (segmented_signal[:int(max_seq_len)])
            
        segmented_signal = np.array(segmented_signal)
        
        if(segmented_signal.shape != (1280,1)):    
            segmented_signal = np.reshape(segmented_signal,(1280,1))
            
        if (normalization):             # Z-score normalization
            if abs(np.std(segmented_signal))<1e-6:          # flat line ECG, will cause zero division error
                continue
            segmented_signal = (segmented_signal - np.mean(segmented_signal)) / np.std(segmented_signal)            
              
        #if not np.isnan(segmented_signal).any():                    # checking for nan, this will never happen
            segmented_signals.append(segmented_signal)

    return segmented_signals,r_peaks           # returning the segmented signals and the refined r-peaks

In [ ]:
###### Looping for Reading the Entire Dataset - OpenSet Testing

! mkdir './5_Beat_Ecg_MITBIH_1' # Generating major database of datasets
current_index = 1

mit_dbs_path = '../input/mitbih-database'
for i in range(0,96,2): # Loop over all the files
    print(i)
    print(np.sort(os.listdir(mit_dbs_path))[i])
    rec_file_path = os.path.join(mit_dbs_path,str(np.sort(os.listdir(mit_dbs_path))[i])) # Path Selection
    attr_file_path = os.path.join(mit_dbs_path,str(np.sort(os.listdir(mit_dbs_path))[i+1])) # Path Selction
    
    signal_current = data_read_mit(rec_file_path) # Current ECG Signal
    r_peaks_current = feature_extractor(attr_file_path) # R-peaks
    seg_signal_current,new_r_peaks = (segmentSignals(signal_current,list(r_peaks_current))) # Segmented ECG Signals
    #seg_signal_current = np.array(seg_signal_current)
    #print(seg_signal_current.shape[0])
    
    if(i == 48):
        current_index = current_index-1
        current_storage_path = './5_Beat_Ecg_MITBIH_1'+'/person'+str(current_index)
        #Path(current_storage_path).mkdir(parents=True, exist_ok=True)
    
        for j in range(len(seg_signal_current)):
            file_name_current = current_storage_path+'/Extra'+str(j)
            np.savez_compressed(file_name_current,seg_signal_current[j])
            
        current_index = current_index+1
    
    else:
        current_storage_path = './5_Beat_Ecg_MITBIH_1'+'/person'+str(current_index)
        Path(current_storage_path).mkdir(parents=True, exist_ok=True)
    
        for j in range(len(seg_signal_current)):
            file_name_current = current_storage_path+'/'+str(j)
            np.savez_compressed(file_name_current,seg_signal_current[j])
            
        current_index = current_index+1

In [ ]:
###### Creation of Numpy Arrays
##### Defining Essentials
data_folder = './5_Beat_Ecg_MITBIH_1'
X_train = []
X_dev = []
y_train = []     
y_dev = []

##### Looping Over to populate the dataset
for index,sub_data_folder in enumerate(np.sort(os.listdir(data_folder))):
    sub_data_folder_path = os.path.join(data_folder,sub_data_folder)
    
    #if(index <= 33):
          
    for idx,item in enumerate(np.sort(os.listdir(sub_data_folder_path))): # Looping Over a person's folder
        item_path = os.path.join(sub_data_folder_path,item)

        #### Train on Past Test on Present (TPTP)
        if(idx <= int(np.round(len(os.listdir(sub_data_folder_path))))):
            X_train.append(np.load(item_path,allow_pickle=True)['arr_0'])
            y_train.append(index+89) # Setting the Value of class label after 89 examples of ECG1D Dataset for CD_EMP
            
    print('Person'+' '+str(index+1)+'s data taken')
    
##### Creation of Numpy Arrays
X_train_CD_train_MITBIH = np.array(X_train)
y_train_CD_train_MITBIH = np.array(y_train)

##### Testing arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

print(np.max(y_train))
print(np.min(y_train))

## PTB Dataset

In [ ]:
###### Constants
FS = 360
W_LEN = 256
W_LEN_1_4 = 256 // 4
W_LEN_3_4 = 3 * (256 // 4)

###### Function to Segment Signals

def segmentSignals(signal, r_peaks_annot, normalization=True, person_id= None, file_id=None):
    
    """
    Segments signals based on the detected R-Peak
    Args:
        signal (numpy array): input signal
        r_peaks_annot (int []): r-peak locations.
        normalization (bool, optional): apply z-normalization or not? . Defaults to True.
        person_id ([type], optional): [description]. Defaults to None.
        file_id ([type], optional): [description]. Defaults to None.
    Returns:
            [tuple(numpy array,numpy array)]: segmented signals and refined r-peaks
    """
    def refine_rpeaks(signal, r_peaks):
        """
        Refines the detected R-peaks. If the R-peak is slightly shifted, this assigns the 
        highest point R-peak.
        Args:
            signal (numpy array): input signal
            r_peaks (int []): list of detected r-peaks
        Returns:
            [numpy array]: refined r-peaks
        """
        r_peaks2 = np.array(r_peaks)            # make a copy
        for i in range(len(r_peaks)):
            r = r_peaks[i]          # current R-peak
            small_segment = signal[max(0,r-100):min(len(signal),r+100)]         # consider the neighboring segment of R-peak
            r_peaks2[i] = np.argmax(small_segment) - 100 + r_peaks[i]           # picking the highest point
            r_peaks2[i] = min(r_peaks2[i],len(signal))                          # the detected R-peak shouldn't be outside the signal
            r_peaks2[i] = max(r_peaks2[i],0)                                    # checking if it goes before zero    
        return r_peaks2                     # returning the refined r-peak list
    
    segmented_signals = []                      # array containing the segmented beats
    
    r_peaks = np.array(r_peaks_annot)

    r_peaks = refine_rpeaks(signal, r_peaks)
    skip_len = 5 # Parameter to specify number of r_peaks in one signal
    max_seq_len = 1280 # Parameter to specify maximum sequence length
    
    for r_curr in range(0,int(r_peaks.shape[0]-(skip_len-1)),skip_len):
        if ((r_peaks[r_curr]-W_LEN_1_4)<0) or ((r_peaks[r_curr+(skip_len-1)]+W_LEN_3_4)>=len(signal)):           # not enough signal to segment
            continue
        segmented_signal = np.array(signal[r_peaks[r_curr]-W_LEN_1_4:r_peaks[r_curr+(skip_len-1)]+W_LEN_3_4])        # segmenting a heartbeat
        segmented_signal = list(segmented_signal)
        #print(segmented_signal.shape)
        
        if(len(segmented_signal) < 1280):
            for m in range(int(1280-len(segmented_signal))): # Zero Padding
                segmented_signal.append(0)
        else:
            segmented_signal = (segmented_signal[:int(max_seq_len)])
            
        segmented_signal = np.array(segmented_signal)
        
        if(segmented_signal.shape != (1280,1)):    
            segmented_signal = np.reshape(segmented_signal,(1280,1))
            
        if (normalization):             # Z-score normalization
            if abs(np.std(segmented_signal))<1e-6:          # flat line ECG, will cause zero division error
                continue
            segmented_signal = (segmented_signal - np.mean(segmented_signal)) / np.std(segmented_signal)            
              
        #if not np.isnan(segmented_signal).any():                    # checking for nan, this will never happen
            segmented_signals.append(segmented_signal)

    return segmented_signals,r_peaks           # returning the segmented signals and the refined r-peaks

###### Function to Read Records

def read_rec(rec_path):

    """ 
    Function to read record and return Segmented Signals

    INPUTS:-
    1) rec_path : Path of the Record

    OUTPUTS:-
    1) seg_sigs : Final Segmented Signals

    """
    number_of_peaks = 5 # For extracting the required number of peaks                                    
    full_rec = (wfdb.rdrecord(rec_path)).p_signal[:,0] # Entire Record - Taking Signal from Lead-1

    f = PchipInterpolator(np.arange(int(full_rec.shape[0])),full_rec) # Fitting Interpolation Function
    num_samples = int(full_rec.shape[0]) # Total Samples in 1000Hz Signal
    num_samples_final = int(num_samples*(360/1000))
    x_samp = (np.arange(num_samples)*(1000/360))[:num_samples_final] # Fixing Interpolation Input Values
    full_rec_interp = f(x_samp)  # Intepolating Values 
    
    r_peaks_init = ecg.hamilton_segmenter(full_rec_interp,360)[0] # R-Peak Segmentation and input is the signal frequency of 500Hz in this case
    final_peak_index = r_peaks_init[int(r_peaks_init.shape[0] - int((r_peaks_init.shape[0]%number_of_peaks)))-1]
    r_peaks_final = r_peaks_init[:final_peak_index] # Final Number of R_Peaks
    full_rec_final = full_rec_interp[:int(r_peaks_final[-1]+W_LEN)] # Final Sequence
    seg_sigs, r_peaks_ref = segmentSignals(full_rec_final,list(r_peaks_final)) # Final Signal Segmentation

    return seg_sigs

In [ ]:
###### Extracting List of the Elements with Two Sessions 
dir = '../input/ptb-dataset/ptb-diagnostic-ecg-database-1.0.0'
total_index = 0
#subjects_with_two = []
subjects = []

for item in np.sort(os.listdir(dir)):
    #print('----------------------------------')
    #print(item)
    dir_sub = os.path.join(dir,item)
    if(os.path.isdir(dir_sub)):
        #ubjects.append(item)
        #print(len(os.listdir(dir_sub))//3)
        if(len(os.listdir(dir_sub))//3 >= 2):
            subjects.append(item)
            #total_index = total_index+1   
            #print(item)
        #    subjects_with_two.append(item)
    #print('----------------------------------')

#print(total_index)
#print(subjects_with_two)

#subjects = shuffle(subjects)[:100] # Shuffling and Taking 100 Subjects 
#print(subjects)

print(subjects)
print(len(subjects))

In [ ]:
###### Creation of Numpy Arrays 
main_dir = '../input/ptb-dataset/ptb-diagnostic-ecg-database-1.0.0'

X_train = []
X_dev = []
y_train = []
y_dev = []

current_index = 0

#for person_index,person_folder in enumerate(list(np.sort(os.listdir(main_dir)))):
 
for person_folder in np.sort(subjects):
    
    #if(current_index <= 231): # Taking 80% of the subjects for training 
    
    person_folder_path = os.path.join(main_dir,person_folder)
    person_folder_items = (list(np.sort(os.listdir(person_folder_path))))

    for file_idx in range(0,len(person_folder_items),3):
        file_path_list = str((os.path.join(person_folder_path,person_folder_items[file_idx])))
        file_num = file_idx//3

        rec_path = ''
        for item_index in range(0,(len(file_path_list)-4)):
            rec_path = rec_path+str(file_path_list[item_index])

        seg_signal_current = read_rec(rec_path) # Extracting Records

        # Division across Time
        #for k in range(len(seg_signal_current)):

        #    if(k <= np.round(len(seg_signal_current)*0.5)):
        #        X_train.append(seg_signal_current[k])
        #        y_train.append(current_index)

        #    else:
        #        X_dev.append(seg_signal_current[k])
        #        y_dev.append(current_index)

        if(file_num == 0):
            for k in range(len(seg_signal_current)):
                X_train.append(seg_signal_current[k])
                y_train.append(current_index)
                
        if(file_num == 1):
            for k in range(len(seg_signal_current)):
                X_dev.append(seg_signal_current[k])
                y_dev.append(current_index)

    current_index = current_index+1
    print('Processed for Person - '+str(current_index))

##### Creation of Numpy Arrays
print(np.array(X_train).shape)
print(np.array(y_train).shape)
print(np.array(X_dev).shape)
print(np.array(y_dev).shape)

print(np.max(y_train))
print(np.min(y_train))

##### Shuffling Arrays
X_train,y_train = shuffle(X_train,y_train)
X_dev,y_dev = shuffle(X_dev,y_dev)

## ECG-1D Dataset

In [ ]:
####### Dataset Creation 

###### Constants
FS = 360
W_LEN = 256
W_LEN_1_4 = 256 // 4
W_LEN_3_4 = 3 * (256 // 4)

###### Function to Read a Record
def read_rec(rec_path):

    """ 
    Function to read record and return Segmented Signals

    INPUTS:-
    1) rec_path : Path of the Record

    OUTPUTS:-
    1) seg_sigs : Final Segmented Signals

    """
    number_of_peaks = 2 # For extracting the required number of peaks                                    
    full_rec = (wfdb.rdrecord(rec_path)).p_signal[:,1] # Entire Record

    f = PchipInterpolator(np.arange(10000),full_rec) # Fitting Interpolation Function
    x_samp = (np.arange(10000)*(500/360))[:7200] # Fixing Interpolation Input Values
    full_rec_interp = f(x_samp)  # Intepolating Values 
    r_peaks_init = ecg.hamilton_segmenter(full_rec_interp,360)[0] # R-Peak Segmentation and input is the signal frequency of 500Hz in this case
    final_peak_index = r_peaks_init[int(r_peaks_init.shape[0] - int((r_peaks_init.shape[0]%number_of_peaks)))-1]
    r_peaks_final = r_peaks_init[:final_peak_index] # Final Number of R_Peaks
    full_rec_final = full_rec_interp[:int(r_peaks_final[-1]+W_LEN)] # Final Sequence
    seg_sigs, r_peaks_ref = segmentSignals(full_rec_final,list(r_peaks_final)) # Final Signal Segmentation

    return seg_sigs # Returning the Ouput of the Signal Segmentation

###### Function to Segment Signals

##### Function
def segmentSignals(signal, r_peaks_annot, normalization=True, person_id= None, file_id=None):
    
    """
    Segments signals based on the detected R-Peak
    Args:
        signal (numpy array): input signal
        r_peaks_annot (int []): r-peak locations.
        normalization (bool, optional): apply z-normalization or not? . Defaults to True.
        person_id ([type], optional): [description]. Defaults to None.
        file_id ([type], optional): [description]. Defaults to None.
    Returns:
            [tuple(numpy array,numpy array)]: segmented signals and refined r-peaks
    """
    def refine_rpeaks(signal, r_peaks):
        """
        Refines the detected R-peaks. If the R-peak is slightly shifted, this assigns the 
        highest point R-peak.
        Args:
            signal (numpy array): input signal
            r_peaks (int []): list of detected r-peaks
        Returns:
            [numpy array]: refined r-peaks
        """
        r_peaks2 = np.array(r_peaks)            # make a copy
        for i in range(len(r_peaks)):
            r = r_peaks[i]          # current R-peak
            small_segment = signal[max(0,r-100):min(len(signal),r+100)]         # consider the neighboring segment of R-peak
            r_peaks2[i] = np.argmax(small_segment) - 100 + r_peaks[i]           # picking the highest point
            r_peaks2[i] = min(r_peaks2[i],len(signal))                          # the detected R-peak shouldn't be outside the signal
            r_peaks2[i] = max(r_peaks2[i],0)                                    # checking if it goes before zero    
        return r_peaks2                     # returning the refined r-peak list
    
    segmented_signals = []                      # array containing the segmented beats
    
    r_peaks = np.array(r_peaks_annot)

    r_peaks = refine_rpeaks(signal, r_peaks)
    skip_len = 5 # Parameter to specify number of r_peaks in one signal
    max_seq_len = 1280 # Parameter to specify maximum sequence length
    
    for r_curr in range(0,int(r_peaks.shape[0]-(skip_len-1)),skip_len):
        if ((r_peaks[r_curr]-W_LEN_1_4)<0) or ((r_peaks[r_curr+(skip_len-1)]+W_LEN_3_4)>=len(signal)):           # not enough signal to segment
            continue
        segmented_signal = np.array(signal[r_peaks[r_curr]-W_LEN_1_4:r_peaks[r_curr+(skip_len-1)]+W_LEN_3_4])        # segmenting a heartbeat
        segmented_signal = list(segmented_signal)
        #print(segmented_signal.shape)
        
        if(len(segmented_signal) < 1280):
            for m in range(int(1280-len(segmented_signal))): # Zero Padding
                segmented_signal.append(0)
        else:
            segmented_signal = (segmented_signal[:int(max_seq_len)])
            
        segmented_signal = np.array(segmented_signal)
        
        if(segmented_signal.shape != (1280,1)):    
            segmented_signal = np.reshape(segmented_signal,(1280,1))
            
        if (normalization):             # Z-score normalization
            if abs(np.std(segmented_signal))<1e-6:          # flat line ECG, will cause zero division error
                continue
            segmented_signal = (segmented_signal - np.mean(segmented_signal)) / np.std(segmented_signal)            
              
        #if not np.isnan(segmented_signal).any():                    # checking for nan, this will never happen
            segmented_signals.append(segmented_signal)

    return segmented_signals,r_peaks           # returning the segmented signals and the refined r-peaks

In [ ]:
###### Numpy Array Creation 
path_to_dir = '../input/ecg1d/ecg-id-database-1.0.0'
total_folders = 90
current_index = 0

X_train = []
X_dev = []
y_train = []
y_dev = []

#for item in subjects_with_two:
for i in range(2,92):

    if(i != 75):

        print(i-1)
        folder_path = os.path.join(path_to_dir,np.sort(os.listdir(path_to_dir))[i]) # Path Selection
        #items_in_folder = int(len(folder_path)//3)
        #current_storage_path = './5_Beat_Ecg_ECG1D'+'/person'+str(current_index)

        #for j in os.listdir(item):

        for j in range(2):

            rec_path = folder_path+'/'+'rec'+'_'+str(j+1) # Path to Record
            seg_signal_current = read_rec(rec_path)

            if(j == 0):
                for k in range(len(seg_signal_current)):
                    #file_name_current = current_storage_path+'/'+str(j)+'_/'+str(k)
                    #np.savez_compressed(file_name_current,seg_signal_current[k])
                    X_train.append(seg_signal_current[k])
                    y_train.append(current_index)

            if(j == 1):
                for k in range(len(seg_signal_current)):
                    #file_name_current = current_storage_path+'/'+str(j)+'_/'+str(k)
                    #np.savez_compressed(file_name_current,seg_signal_current[k])
                    X_train.append(seg_signal_current[k])
                    y_train.append(current_index)

        current_index = current_index+1

###### Creation of Numpy Arrays
X_train_CD_train_ECG1D = np.array(X_train)
y_train_CD_train_ECG1D = np.array(y_train)

###### Checking Shape of the Arrays
print(np.array(X_train).shape)
print(np.array(y_train).shape)

print(np.max(y_train))
print(np.min(y_train))